In [15]:
# Imports
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset
from torchvision import transforms, datasets
from PIL import Image
import os
import zipfile
import torch.optim as optim
import torch.nn as nn
import numpy as np
from sklearn.model_selection import KFold
import random

class MixedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __getitem__(self, index):
        img, label = self.dataset[index]
        if isinstance(img, torch.Tensor):
            # If it's already a tensor, just normalize it
            img = transforms.Normalize((0.5,), (0.5,))(img)
        elif isinstance(img, Image.Image):
            # If it's a PIL Image, apply the full transform
            img = self.transform(img)
        else:
            raise TypeError(f"Unexpected type: {type(img)}")
        return img, label

    def __len__(self):
        return len(self.dataset)

# Data Loading and Augmentation
class TensorDataset(Dataset):
    def __init__(self, tensors, transform=None):
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]
        y = self.tensors[1][index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

# Data Loading and Augmentation
def add_noise(image, noise_factor=0.5):
    noise = torch.randn_like(image) * noise_factor
    noisy_image = image + noise
    return torch.clamp(noisy_image, 0., 1.)

class AugmentedDataset(Dataset):
    def __init__(self, original_dataset, transform=None):
        self.original_dataset = original_dataset
        self.transform = transform

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        image, label = self.original_dataset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Load experimental data
def load_all_experimental_data(test_digits_folder):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    participant_data = {}

    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    for filename in os.listdir(test_digits_folder):
        if filename.endswith('.zip') and filename.startswith('experiment_results_participant'):
            participant_number = int(filename.split('participant')[1].split('.')[0])
            zip_filepath = os.path.join(test_digits_folder, filename)

            participant_train_images = []
            participant_train_labels = []
            participant_test_images = []
            participant_test_labels = []

            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                for img_filename in zip_ref.namelist():
                    if img_filename.endswith('.png'):
                        with zip_ref.open(img_filename) as file:
                            img = Image.open(file).convert('L')
                            img_tensor = transform(img)
                            
                            digit = int(img_filename.split('_')[0])
                            
                            if 'composite' in img_filename:
                                test_images.append(img_tensor)
                                test_labels.append(digit)
                                participant_test_images.append(img_tensor)
                                participant_test_labels.append(digit)
                            else:
                                train_images.append(img_tensor)
                                train_labels.append(digit)
                                participant_train_images.append(img_tensor)
                                participant_train_labels.append(digit)

            participant_data[participant_number] = {
                'train': (torch.stack(participant_train_images), torch.tensor(participant_train_labels)),
                'test': (torch.stack(participant_test_images), torch.tensor(participant_test_labels))
            }

    return (torch.stack(train_images), torch.tensor(train_labels), 
            torch.stack(test_images), torch.tensor(test_labels),
            participant_data)

# Load experimental data
exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')

# Load and augment MNIST data
def load_augmented_mnist():
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
    ])
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    # Create noisy versions
    noisy_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: add_noise(x, noise_factor=0.5)),
    ])
    noisy_mnist_train = AugmentedDataset(mnist_train, noisy_transform)
    noisy_mnist_test = AugmentedDataset(mnist_test, noisy_transform)

    # Create inverted versions
    invert_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: 1 - x),
    ])
    inverted_mnist_train = AugmentedDataset(mnist_train, invert_transform)
    inverted_mnist_test = AugmentedDataset(mnist_test, invert_transform)

    # Combine datasets
    combined_train = ConcatDataset([mnist_train, noisy_mnist_train, inverted_mnist_train])
    combined_test = ConcatDataset([mnist_test, noisy_mnist_test, inverted_mnist_test])

    return combined_train, combined_test

In [2]:
# Model Definition
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    best_val_acc = 0
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')
        
        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                outputs_val = model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_acc = 100 * correct_val / total_val
        print(f'Validation Accuracy: {val_acc:.2f}%')
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
    
    return model

In [3]:
def k_fold_cross_validation(dataset, num_folds=5, num_epochs=50, patience=5):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_index, val_index) in enumerate(kf.split(range(len(dataset)))):
        print(f"Fold {fold + 1}/{num_folds}")
        train_subset = torch.utils.data.Subset(dataset, train_index)
        val_subset = torch.utils.data.Subset(dataset, val_index)

        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

        model = LeNet5_16x16(num_classes=10)
        
        # Calculate class weights for weighted cross-entropy
        labels = torch.tensor([dataset[i][1] for i in train_index])
        class_counts = torch.bincount(labels)
        class_weights = 1. / class_counts.float()
        class_weights = class_weights / class_weights.sum()
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience)

        # Evaluate on validation set
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                outputs_val = model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_accuracy = 100 * correct_val / total_val
        fold_results.append(val_accuracy)
        print(f'Fold {fold + 1} Validation Accuracy: {val_accuracy:.2f}%')

    return fold_results

In [4]:
if __name__ == "__main__":
    # Load experimental data
    exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')
    exp_dataset = TensorDataset(exp_train_images, exp_train_labels)

    # Load augmented MNIST data
    mnist_train, mnist_test = load_augmented_mnist()

    # Combine experimental data with augmented MNIST data
    combined_dataset = ConcatDataset([exp_dataset, mnist_train])

    # Wrap the combined dataset with MixedDataset
    mixed_dataset = MixedDataset(combined_dataset)

    # Perform K-Fold Cross Validation
    fold_results = k_fold_cross_validation(mixed_dataset, num_folds=5, num_epochs=50, patience=5)

    print(f"Average Validation Accuracy across folds: {np.mean(fold_results):.2f}%")

    # Train final model on all data
    final_train_loader = DataLoader(mixed_dataset, batch_size=32, shuffle=True)
    final_val_loader = DataLoader(MixedDataset(mnist_test), batch_size=32, shuffle=False)

    final_model = LeNet5_16x16(num_classes=10)
    labels = torch.tensor([y for _, y in mixed_dataset])
    class_counts = torch.bincount(labels)
    class_weights = 1. / class_counts.float()
    class_weights = class_weights / class_weights.sum()
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(final_model.parameters(), lr=0.001)

    final_model = train_model(final_model, final_train_loader, final_val_loader, criterion, optimizer, num_epochs=50, patience=5)

    # Save the final model
    torch.save(final_model.state_dict(), "lenet5_trained_model_heavy.pth")
    print("Final model saved as lenet5_trained_model_heavy.pth")

Fold 1/5


TypeError: pic should be Tensor or ndarray. Got <class 'PIL.Image.Image'>.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from torchvision import datasets, transforms
import numpy as np
from sklearn.model_selection import KFold

# Define LeNet5 architecture modified for 16x16 images with dropout
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = torch.tanh(self.fc1(out))
        out = torch.tanh(self.fc2(out))
        out = self.fc3(out)
        return out

# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert MNIST datasets to TensorDataset
mnist_train_data = torch.stack([img for img, _ in mnist_train])
mnist_train_labels = torch.tensor([label for _, label in mnist_train])
mnist_train_tensor = TensorDataset(mnist_train_data, mnist_train_labels)

mnist_test_data = torch.stack([img for img, _ in mnist_test])
mnist_test_labels = torch.tensor([label for _, label in mnist_test])
mnist_test_tensor = TensorDataset(mnist_test_data, mnist_test_labels)

# Combine MNIST and experimental data
exp_train_dataset = TensorDataset(exp_train_images, exp_train_labels)
val_dataset = TensorDataset(val_images, val_labels)

combined_train = ConcatDataset([mnist_train_tensor, exp_train_dataset])
combined_val = ConcatDataset([mnist_test_tensor, val_dataset])

# Calculate class weights
def calculate_class_weights(combined_dataset):
    all_labels = []
    for dataset in combined_dataset.datasets:
        if isinstance(dataset, TensorDataset):
            all_labels.extend(dataset.tensors[1].tolist())
        else:
            all_labels.extend([label for _, label in dataset])
    
    labels = torch.tensor(all_labels)
    class_counts = torch.bincount(labels)
    class_weights = 1. / class_counts.float()
    return class_weights / class_weights.sum()

class_weights = calculate_class_weights(combined_train)

# Fine-tuning function for LeNet5
def fine_tune_lenet(model, train_loader, val_loader, class_weights, num_epochs=50, save_path="lenet5_model.pth"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    class_weights = class_weights.to(device)
    
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    best_val_loss = float('inf')
    patience = 10
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                loss_val = criterion(outputs_val, val_labels)
                val_loss += loss_val.item()
                
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping")
                break
    
    return model

# K-fold cross-validation
def k_fold_cross_validation(combined_dataset, num_folds=5, num_epochs=50):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_index, val_index) in enumerate(kf.split(range(len(combined_dataset)))):
        print(f"Fold {fold + 1}/{num_folds}")
        train_subset = torch.utils.data.Subset(combined_dataset, train_index)
        val_subset = torch.utils.data.Subset(combined_dataset, val_index)

        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

        model = LeNet5_16x16()
        class_weights = calculate_class_weights(train_subset)
        
        trained_model = fine_tune_lenet(model, train_loader, val_loader, class_weights, num_epochs=num_epochs, save_path=f"lenet5_model_fold_{fold+1}.pth")

        # Evaluate on validation set
        trained_model.eval()
        correct_val = 0
        total_val = 0
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = trained_model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_accuracy = 100 * correct_val / total_val
        fold_results.append(val_accuracy)
        print(f'Fold {fold + 1} Validation Accuracy: {val_accuracy:.2f}%')

    return fold_results

# Perform K-fold cross-validation
fold_results = k_fold_cross_validation(combined_train, num_folds=5, num_epochs=50)
print(f"Average Validation Accuracy across folds: {np.mean(fold_results):.2f}%")

# Train final model on all data
final_train_loader = DataLoader(combined_train, batch_size=64, shuffle=True)
final_val_loader = DataLoader(combined_val, batch_size=64, shuffle=False)

final_model = LeNet5_16x16()
final_class_weights = calculate_class_weights(combined_train)

trained_final_model = fine_tune_lenet(final_model, final_train_loader, final_val_loader, final_class_weights, num_epochs=50, save_path="lenet5_final_model.pth")

# Evaluate on test set
def evaluate_lenet(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    correct_test = 0
    total_test = 0
    
    with torch.no_grad():
        for images_batch_test, labels_batch_test in test_loader:
            images_batch_test, labels_batch_test = images_batch_test.to(device), labels_batch_test.to(device)
            outputs_test = model(images_batch_test)
            _, predicted_test_classifications = torch.max(outputs_test.data, 1)
            
            total_test += labels_batch_test.size(0)
            correct_test += (predicted_test_classifications == labels_batch_test).sum().item()
    
    print(f'Accuracy of the network on final test images: {100 * correct_test / total_test:.2f}%')

# Evaluate on final test set
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
evaluate_lenet(trained_final_model, test_loader)

Fold 1/5


TypeError: pic should be Tensor or ndarray. Got <class 'PIL.Image.Image'>.

In [6]:
print(f"Type of first item in dataset: {type(dataset[0][0])}")
print(f"Shape of first item in dataset: {dataset[0][0].shape if isinstance(dataset[0][0], torch.Tensor) else 'Not a tensor'}")

NameError: name 'dataset' is not defined

In [16]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset
from torchvision import transforms, datasets
from PIL import Image
import os
import zipfile
import torch.optim as optim
import torch.nn as nn
import numpy as np
from sklearn.model_selection import KFold, train_test_split
import random

# Data Loading and Augmentation
def add_noise(image, noise_factor=0.5):
    noise = torch.randn_like(image) * noise_factor
    noisy_image = image + noise
    return torch.clamp(noisy_image, 0., 1.)

class AugmentedDataset(Dataset):
    def __init__(self, original_dataset, transform=None):
        self.original_dataset = original_dataset
        self.transform = transform

    def __len__(self):
        return len(self.original_dataset)

    def __getitem__(self, idx):
        image, label = self.original_dataset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def load_augmented_mnist():
    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
    ])
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    # Create noisy versions
    noisy_transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: add_noise(x, noise_factor=0.5)),
    ])
    noisy_mnist_train = AugmentedDataset(mnist_train, noisy_transform)
    noisy_mnist_test = AugmentedDataset(mnist_test, noisy_transform)

    # Create inverted versions
    invert_transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: 1 - x),
    ])
    inverted_mnist_train = AugmentedDataset(mnist_train, invert_transform)
    inverted_mnist_test = AugmentedDataset(mnist_test, invert_transform)

    # Combine datasets
    combined_train = ConcatDataset([mnist_train, noisy_mnist_train, inverted_mnist_train])
    combined_test = ConcatDataset([mnist_test, noisy_mnist_test, inverted_mnist_test])

    return combined_train, combined_test

# Model Definition
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [17]:
# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((16, 16)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load and prepare data
exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')
exp_train_dataset = TensorDataset(exp_train_images, exp_train_labels)
exp_test_dataset = TensorDataset(exp_test_images, exp_test_labels)

mnist_train, mnist_test = load_augmented_mnist()

# Combine all data
all_data = ConcatDataset([mnist_train, mnist_test, exp_train_dataset, exp_test_dataset])

# K-Fold Cross-validation
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Training function
def train_model(model, train_loader, val_loader, num_epochs=30):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)
    
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                loss_val = criterion(outputs_val, val_labels)
                val_loss += loss_val.item()
                
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
    
    return model

# K-Fold Cross-validation training
for fold, (train_ids, val_ids) in enumerate(kfold.split(all_data)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    
    train_loader = DataLoader(all_data, batch_size=32, sampler=train_subsampler)
    val_loader = DataLoader(all_data, batch_size=32, sampler=val_subsampler)
    
    model = LeNet5_16x16()
    train_model(model, train_loader, val_loader)

# Load the best model and evaluate on the test set
best_model = LeNet5_16x16()
best_model.load_state_dict(torch.load("best_model.pth"))

# Create a test loader
test_loader = DataLoader(exp_test_dataset, batch_size=32, shuffle=False)

# Evaluate function (assuming it's defined as before)
evaluate_lenet(best_model, test_loader)

FOLD 0
--------------------------------


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>